# Assignment 03 (Individual) - Due Thursday, November 16 at Midnight

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.


*Deadline*: Thursday, November 16 at Midnight  

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [data sets](https://tweetsets.library.gwu.edu/datasets) during a game of the MLB World Series featuring the Los Angeles Dodgers and Houston Astros.  This first file tells you a little bit about how it was gathered:

#### First make sure you are working from the right working directory

In [1]:
!pwd

/home/ubuntu/notebooks/Indi Assignment


#### This below file provides context and detail information on the files we are going to work with
#### in this assignment

In [2]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt

--2023-11-07 21:49:22--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.128.32, 54.231.192.200, 16.182.64.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.128.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920 (1.9K) [text/plain]
Saving to: ‘9670f3399f774789b7c3e18975d25611-README.txt’

9670f3399f774789b7c 100%[===================>]   1.88K  --.-KB/s    in 0s      

2023-11-07 21:49:22 (95.4 MB/s) - ‘9670f3399f774789b7c3e18975d25611-README.txt’ saved [1920/1920]



In [3]:
!cat 9670f3399f774789b7c3e18975d25611-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: 9670f3399f774789b7c3e18975d25611
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2017, 11:21:04 p.m. EDT
Deduplicate: No

COLLECTION INFORMATION
Collection name: test set for world series
Collection id: 34e3f7460b5c4df09d64a1e61fd81238
Collection set: mlb-test (collection set id d6e8c27b1bc942e78790aa55a82b3a7a)
Harvest type: Twitter filter
Collection description: running for just one hour, just for fun.

Harvest options:
Media: No
Web resources: No

Seeds:
* Track: dodgers,astros - Active

Change log:

Change to test set for world series (collection) on Oct. 30, 2017, 10:59:56 p.m. EDT by dchud:
* is_active: "True" changed to "False"

Change to test set for world series (collection) on Oct. 30, 2017, 10:58:51 p.m. EDT by dchud:
* is_on: "True" changed to "False"

Change to test set for world series (collection) on Oct. 29, 2017, 8:01:24 p.m. EDT by dch

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 
You should not need to know anything about baseball to complete this assignment.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 
### Q1.1 - Upload the above files to your instance using `wget`.  Verify the file sizes using the command line. 

Each file should contain exactly 100,000 tweets.  

*Note*: you are required to use all files.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [4]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json

--2023-11-07 21:50:26--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.162.165, 52.216.214.144, 52.217.132.200, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.162.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  66.1MB/s    in 8.8s    

2023-11-07 21:50:35 (64.3 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



In [5]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json

--2023-11-07 21:51:01--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.170.160, 52.217.225.240, 54.231.139.8, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.170.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545081593 (520M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_006.json’

9670f3399f774789b7c 100%[===================>] 519.83M  58.8MB/s    in 8.5s    

2023-11-07 21:51:09 (61.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_006.json’ saved [545081593/545081593]



#### Check the files have exactly 100,000 tweets using the command line

In [10]:
!wc -l 9670f3399f774789b7c3e18975d25611_003.json

100000 9670f3399f774789b7c3e18975d25611_003.json


In [11]:
!wc -l 9670f3399f774789b7c3e18975d25611_006.json

100000 9670f3399f774789b7c3e18975d25611_006.json


For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [13]:
!cat *.json | shuf -n 1 > example-tweet.json

In [14]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "quote_count": 0,
  "contributors": null,
  "truncated": false,
  "text": "No argument that a glaring disparity has been identified. https://t.co/KPUPOaU7HP",
  "is_quote_status": true,
  "in_reply_to_status_id": null,
  "reply_count": 0,
  "id": 924827857280536576,
  "favorite_count": 0,
  "entities": {
    "user_mentions": [],
    "symbols": [],
    "hashtags": [],
    "urls": [
      {
        "url": "https://t.co/KPUPOaU7HP",
        "indices": [
          58,
          81
        ],
        "expanded_url": "https://twitter.com/jaysonst/status/924826054790995968",
        "display_url": "twitter.com/jaysonst/statu\u2026"
      }
    ]
  },
  "quoted_status_id": 924826054790995968,
  "retweeted": false,
  "coordinates": null,
  "timestamp_ms": "1509331118303",
  "quoted_status": {
    "quote_count": 32,
    "contributors": null,
    "truncated": false,
    "text": "Astros bullpen has now allowed 30 ER in this postseason (in 48 IP). Dodgers bullpen has allowed 10 (in 47.1). No fu

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  
If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [1]:
import os

In [2]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext

In [6]:
spark = SparkContext(appName='project-03')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/17 03:52:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

<SparkContext master=local[*] appName=project-03>

In [8]:
from pyspark import SQLContext

In [9]:
sqlc = SQLContext(spark)

/usr/local/lib/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
sqlc

In [28]:
!rm -f example-tweet.json

We are removing the exmpale tweet here to avoid the confusion while taking the tweet count below and also for further questions to not duplicate this exmaple as it is the data that is already existing
in the json files

In [11]:
tweets = sqlc.read.json("*.json")

23/11/17 03:53:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Verify that Spark has loaded the same number of tweets you saw before:

**Answer** Yes, spark has also loaded the same number of tweets

In [13]:
%time tweets.count()

CPU times: user 5.24 ms, sys: 286 µs, total: 5.53 ms
Wall time: 2.68 s


200000

#### We can see that word count of both JSON files combined is 200000 and it's same as using command line above.

Do you see exactly the same number of tweets in Spark that you saw on the command line? 

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

To issue SQL queries, we need to register a table based on `tweets`:

This is all well and good, but how well did schema inference work?

In [12]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 6 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

In [28]:
tweets.groupBy("lang").agg({"lang": "count"}).orderBy("count(lang)", ascending=False).select('lang',"count(lang)").show(6)

+----+-----------+
|lang|count(lang)|
+----+-----------+
|  en|     173906|
|  es|      15666|
| und|       7579|
|  in|        483|
|  pt|        479|
|  fr|        435|
+----+-----------+
only showing top 6 rows



In [17]:
sqlc.sql("""
    SELECT lang, count(lang) 
    FROM tweets
    group by lang
    ORDER BY count(lang) DESC
""").show(6)

+----+-----------+
|lang|count(lang)|
+----+-----------+
|  en|     173906|
|  es|      15666|
| und|       7579|
|  in|        483|
|  pt|        479|
|  fr|        435|
+----+-----------+
only showing top 6 rows



#### As seen above, 'en' is the most common language used in tweets with 173906 times used and also the top are as seen in the outputs above using both dataframe and SQL methods. We have taken count of language count(lang) here to get the aggregate of users using that particular language and order by descending to get the top languages.


### Q2.2 - Which 5 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [37]:
tweets.select('user.time_zone').groupBy("time_zone").agg({"time_zone": "count"}).orderBy("count(time_zone)", ascending=False).show(5)

+--------------------+----------------+
|           time_zone|count(time_zone)|
+--------------------+----------------+
|Pacific Time (US ...|           35816|
|Central Time (US ...|           32031|
|Eastern Time (US ...|           17780|
|             Arizona|            5199|
|Mountain Time (US...|            4700|
+--------------------+----------------+
only showing top 5 rows



In [38]:
sqlc.sql("""
    SELECT user.time_zone,count(user.time_zone)
    FROM tweets
    group by user.time_zone
    ORDER BY count(user.time_zone) DESC
""").show(5)

+--------------------+---------------------+
|           time_zone|count(user.time_zone)|
+--------------------+---------------------+
|Pacific Time (US ...|                35816|
|Central Time (US ...|                32031|
|Eastern Time (US ...|                17780|
|             Arizona|                 5199|
|Mountain Time (US...|                 4700|
+--------------------+---------------------+
only showing top 5 rows



#### Five most common time zones among users start with Pacific time and end with Mountain Time as seen in the above outputs/results from both dataframe and SQL. We have taken count of time_zone to get the number of users in that partiular time zone and order by count(time_zone) in descending order to get the top time_zones

### Q2.3 - How many tweets mention the Dodgers?  How many mention the Astros?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

In [27]:
sqlc.sql("""
    SELECT count(text)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
""").show()

+-----------+
|count(text)|
+-----------+
|      78522|
+-----------+



In [30]:
sqlc.sql("""
    SELECT count(text)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
""").show()

+-----------+
|count(text)|
+-----------+
|     127041|
+-----------+



In [31]:
sqlc.sql("""
    SELECT count(text)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
    AND LOWER(text) LIKE '%dodgers%'
""").show()

+-----------+
|count(text)|
+-----------+
|      24016|
+-----------+



#### I have used SQL approach to get the tweets that mention Astros or Dodgers or Both because I felt it is easier method for this question to use SQL. I have converted all text into lower case to eliminate case issues while looking for astros or dodgers.
##### Astros are mentioned 127041 times, Dodgers are mentioned 78522 times and the tweets that contain both of them are 24016.

## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Astros and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings. Do not count null time_zones or location.

Hint:  you may use either the time zones or user-specified locations for this question.


**Answer**

In [21]:
sqlc.sql("""
    SELECT user.time_zone, count(user.time_zone)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
    GROUP BY user.time_zone
    ORDER BY count(user.time_zone) DESC
""").show()

+--------------------+---------------------+
|           time_zone|count(user.time_zone)|
+--------------------+---------------------+
|Central Time (US ...|                25187|
|Pacific Time (US ...|                17165|
|Eastern Time (US ...|                12430|
|Mountain Time (US...|                 3392|
|             Arizona|                 2257|
|Atlantic Time (Ca...|                 1700|
|             Caracas|                 1420|
|               Quito|                 1339|
|              Hawaii|                 1107|
|         Mexico City|                 1055|
|     America/Chicago|                  738|
|              Alaska|                  628|
|     Central America|                  431|
|              Bogota|                  312|
|            Brasilia|                  272|
| America/Los_Angeles|                  253|
|           Monterrey|                  233|
|              London|                  215|
|    America/New_York|                  209|
|         

In [20]:
sqlc.sql("""
    SELECT user.time_zone, count(user.time_zone)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
    GROUP BY user.time_zone
    ORDER BY count(user.time_zone) DESC
""").show()

+--------------------+---------------------+
|           time_zone|count(user.time_zone)|
+--------------------+---------------------+
|Pacific Time (US ...|                19435|
|Central Time (US ...|                 6699|
|Eastern Time (US ...|                 5960|
|             Arizona|                 3026|
|Mountain Time (US...|                 1407|
|             Caracas|                 1217|
|         Mexico City|                 1043|
|              Alaska|                  926|
|               Quito|                  853|
|Atlantic Time (Ca...|                  816|
|              Hawaii|                  692|
| America/Los_Angeles|                  335|
|             Tijuana|                  262|
|     Central America|                  254|
|     America/Chicago|                  174|
|              Bogota|                  168|
|               Tokyo|                  155|
|            Brasilia|                  145|
|              London|                  138|
|         

In [22]:
sqlc.sql("""
    SELECT user.time_zone, count(user.time_zone)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%' AND LOWER(text) LIKE '%astros%'
    GROUP BY user.time_zone
    ORDER BY count(user.time_zone) DESC
""").show()

+--------------------+---------------------+
|           time_zone|count(user.time_zone)|
+--------------------+---------------------+
|Pacific Time (US ...|                 4286|
|Central Time (US ...|                 3193|
|Eastern Time (US ...|                 2508|
|Mountain Time (US...|                  570|
|             Arizona|                  554|
|         Mexico City|                  511|
|             Caracas|                  508|
|Atlantic Time (Ca...|                  339|
|               Quito|                  334|
|              Hawaii|                  216|
|     Central America|                  137|
|              Alaska|                  136|
|              Bogota|                  105|
| America/Los_Angeles|                   89|
|     America/Chicago|                   87|
|              London|                   75|
|            Brasilia|                   64|
|           Monterrey|                   60|
|    America/New_York|                   51|
|         

#### Using SQL we found the team mentions astros and dodgers separately as well as combined using time zone. Converted text into lower case to eliminate the case sensitive nature and also used count(time_zone) to get the aggregate of time zone users

### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [39]:
sqlc.sql("""
    SELECT in_reply_to_screen_name, count(in_reply_to_screen_name)
    FROM tweets
    GROUP BY in_reply_to_screen_name
    ORDER BY count(in_reply_to_screen_name) DESC
""").show()

+-----------------------+------------------------------+
|in_reply_to_screen_name|count(in_reply_to_screen_name)|
+-----------------------+------------------------------+
|                 astros|                          2061|
|                Dodgers|                          1569|
|                    MLB|                           464|
|               MLBONFOX|                           113|
|          stephenasmith|                           109|
|          DodgerInsider|                            92|
|          DodgersNation|                            92|
|           JoseAltuve27|                            59|
|                 feSOUL|                            57|
|             JohnLegere|                            55|
|          Nick_Offerman|                            54|
|              adevaldes|                            53|
|                ABREG_1|                            51|
|           ESPN_Beisbol|                            50|
|                trvisXX|      

We have taken the count of in_reply_to_screen_name to get the aggregate count and grouping it by the same column. Order by the count(in_reply_to_screen_name) in descending order to get the top most 


### Q3.3 - Which 12 verified users have the most followers?  Which 12 unverified users have the most followers?

Provide both the screen names (screen_name) and follower counts (followers_count) for each.
Verified users -- use verified == 't'

Discuss your findings.

**Answer**

In [17]:
sqlc.sql("""
    SELECT user.screen_name, max(user.followers_count), user.verified
    FROM tweets
    WHERE user.verified == true
    GROUP BY user.screen_name, user.verified
    ORDER BY max(user.followers_count) DESC
""").show(12)

+--------------+-------------------------+--------+
|   screen_name|max(user.followers_count)|verified|
+--------------+-------------------------+--------+
|       Reuters|                 18937529|    true|
|       FoxNews|                 16272836|    true|
|           ABC|                 12551437|    true|
|washingtonpost|                 11417638|    true|
|           MLB|                  7841255|    true|
|           NPR|                  7289492|    true|
|   BillSimmons|                  6000106|    true|
|     Residente|                  5856019|    true|
|       NBCNews|                  5442705|    true|
|    JohnLegere|                  4630104|    true|
|     ANCALERTS|                  4453229|    true|
|       Milenio|                  4007212|    true|
+--------------+-------------------------+--------+
only showing top 12 rows



Reuters have the highest followers count in verified users while Milenio is in the 12th place

In [23]:
sqlc.sql("""
    SELECT user.screen_name, max(user.followers_count), user.verified
    FROM tweets
    WHERE user.verified == false
    GROUP BY user.verified,user.screen_name
    ORDER BY max(user.followers_count) DESC
""").show(12)

+---------------+-------------------------+--------+
|    screen_name|max(user.followers_count)|verified|
+---------------+-------------------------+--------+
|Daminous_Purity|                   998742|   false|
|        chochos|                   833669|   false|
|  el_carabobeno|                   725952|   false|
|       PAMsLOvE|                   712254|   false|
| mlbtraderumors|                   659851|   false|
|        jilevin|                   568341|   false|
|    sun_das_ill|                   559669|   false|
|   DiegoArcos14|                   544915|   false|
|    TVCDeportes|                   543095|   false|
|       EP_Mundo|                   538525|   false|
|         LALATE|                   516153|   false|
|  piercearrow33|                   503015|   false|
+---------------+-------------------------+--------+
only showing top 12 rows



Daminous_Purity has the highest followers count in unverified users with 998742 followers while piercearrow33 is in the 12th place

We have used user.verified in where condition here to check whether the user if verified or not and grouping by followers count, screen_name and verified. Then we have used order by followers_count in descending order to get the top most count of the followers

### Q4 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", remove at least 12 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  A simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation. 

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Astros fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

You will notice that common words include words like "thisteam" and "earnhistory". I would like you to write two queries to investigate whether those two words are used by the Astros or Dodgers

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [20]:
rdd_top = tweets.rdd.flatMap(lambda r: r['text'].split(' ')) \
        .map(lambda t: (t, 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .takeOrdered(50, key=lambda pair: -pair[1])
rdd_top

[('RT', 115635),
 ('the', 78439),
 ('a', 34723),
 ('Astros', 33868),
 ('to', 33352),
 ('', 29647),
 ('in', 29533),
 ('@astros:', 29038),
 ('Dodgers', 24360),
 ('#WorldSeries', 24328),
 ('for', 20991),
 ('@astros', 18166),
 ('of', 17664),
 ('is', 17099),
 ('I', 16248),
 ('and', 15898),
 ('this', 15561),
 ('The', 14733),
 ('#EarnHistory', 14305),
 ('#Astros', 14012),
 ('@Dodgers', 13856),
 ('game', 13295),
 ('win', 11522),
 ('on', 10849),
 ('@Dodgers:', 10090),
 ('are', 9689),
 ('it', 9623),
 ('that', 9538),
 ('#Dodgers', 9311),
 ('World', 8715),
 ('THE', 8522),
 ('you', 8505),
 ('A', 8259),
 ('de', 7749),
 ('was', 7727),
 ('Game', 7428),
 ('Series', 7404),
 ('have', 7090),
 ('just', 7049),
 ('This', 6968),
 ('5', 6939),
 ('one', 6770),
 ('#ThisTeam', 6436),
 ('with', 6356),
 ("LET'S", 6285),
 ('GAME', 6279),
 ('out', 6076),
 ('dodgers', 6042),
 ('go', 6030),
 ('TIE', 5986)]

Above query is to get the top 50 frequent words used by the users and we can see how many times each word is used in the output

In [16]:
stop_words = ["a", "is", "the", "rt", "and", "to", "of", "in", "for", "i", "this", "The", "", "on", "are", "it", "you", "was", "have", "that", "5"]  

result=tweets.rdd.flatMap(lambda r: r['text'].split(' ')) \
    .filter(lambda t: t.lower() not in stop_words) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(20, key=lambda pair: -pair[1])
result

[('Astros', 33868),
 ('@astros:', 29038),
 ('Dodgers', 24360),
 ('#WorldSeries', 24328),
 ('@astros', 18166),
 ('#EarnHistory', 14305),
 ('#Astros', 14012),
 ('@Dodgers', 13856),
 ('game', 13295),
 ('win', 11522),
 ('@Dodgers:', 10090),
 ('#Dodgers', 9311),
 ('World', 8715),
 ('de', 7749),
 ('Game', 7428),
 ('Series', 7404),
 ('just', 7049),
 ('one', 6770),
 ('#ThisTeam', 6436),
 ('with', 6356)]

We have defined some stop words using the result from previous query and filtered out/removed those from the frequent commmon words and displayed the output

In [17]:
tweets_df = sqlc.createDataFrame(result, ["word", "count"])
tweets_df.show()

tweets_df.createOrReplaceTempView("tweets_words")

+------------+-----+
|        word|count|
+------------+-----+
|      Astros|33868|
|    @astros:|29038|
|     Dodgers|24360|
|#WorldSeries|24328|
|     @astros|18166|
|#EarnHistory|14305|
|     #Astros|14012|
|    @Dodgers|13856|
|        game|13295|
|         win|11522|
|   @Dodgers:|10090|
|    #Dodgers| 9311|
|       World| 8715|
|          de| 7749|
|        Game| 7428|
|      Series| 7404|
|        just| 7049|
|         one| 6770|
|   #ThisTeam| 6436|
|        with| 6356|
+------------+-----+



We have converted the list into a dataframe and then to a view to use it in the following query as a table

In [20]:
dodgers_words = sqlc.sql("""
    SELECT w.word, COUNT(w.word) as count
    FROM tweets t
    JOIN tweets_words w ON t.text LIKE ('%' || w.word || '%')
    WHERE lower(t.text) LIKE '%dodgers%' 
    GROUP BY w.word
    ORDER BY count DESC
""")

dodgers_words.show()

+------------+-----+
|        word|count|
+------------+-----+
|     Dodgers|67403|
|    @Dodgers|26897|
|       World|21872|
|      Series|21395|
|      Astros|18249|
|#WorldSeries|17443|
|          de|12925|
|    #Dodgers|10619|
|   #ThisTeam|10405|
|   @Dodgers:|10093|
|        game| 8202|
|        Game| 5649|
|         win| 5375|
|         one| 4631|
|     #Astros| 4408|
|     @astros| 4384|
|        with| 3139|
|        just| 2056|
|#EarnHistory| 1110|
|    @astros:|  178|
+------------+-----+



In [24]:
astros_words = sqlc.sql("""
    SELECT w.word, COUNT(w.word) as count
    FROM tweets t
    JOIN tweets_words w ON t.text LIKE ('%' || w.word || '%')
    WHERE lower(t.text) LIKE '%astros%' 
    GROUP BY w.word
    ORDER BY count DESC
""")

astros_words.show()

+------------+-----+
|        word|count|
+------------+-----+
|      Astros|63814|
|     @astros|50283|
|    @astros:|29044|
|       World|26983|
|      Series|26650|
|     Dodgers|21902|
|#EarnHistory|20479|
|#WorldSeries|19531|
|          de|16065|
|     #Astros|15905|
|        game|15249|
|         win|15095|
|        Game| 8955|
|         one| 8918|
|        just| 5821|
|    @Dodgers| 5253|
|        with| 4897|
|    #Dodgers| 4693|
|   #ThisTeam| 1202|
|   @Dodgers:|  356|
+------------+-----+



In the above two queries, we joined tables tweets and created table tweets_words and with conditions to check for astros or dodgers. It gave us the count of words where the words are seen along with dodgers or astros

In [25]:
thisteam_dodgers_result = sqlc.sql("""
    SELECT COUNT(*) as count
    FROM tweets
    WHERE text LIKE '%thisteam%'
    AND lower(text) LIKE '%dodgers%' AND LOWER(text) NOT LIKE '%astros%'
""")

thisteam_dodgers_result.show()

+-----+
|count|
+-----+
|  423|
+-----+



In [25]:
thisteam_astros_result = sqlc.sql("""
    SELECT COUNT(*) as count
    FROM tweets
    WHERE text LIKE '%thisteam%'
    AND lower(text) LIKE '%astros%' AND LOWER(text) NOT LIKE '%dodgers%'
""")

thisteam_astros_result.show()

+-----+
|count|
+-----+
|   20|
+-----+



From the above two queries, we can say that dodgers are most likely to use the word thisteam in their tweet

In [26]:
earnhistory_dodgers_result = sqlc.sql("""
    SELECT COUNT(*) as count
    FROM tweets
    WHERE text LIKE '%EarnHistory%'
    AND lower(text) LIKE '%dodgers%' AND LOWER(text) NOT LIKE '%astros%'
""")

earnhistory_dodgers_result.show()

+-----+
|count|
+-----+
|  149|
+-----+



In [27]:
earnhistory_astros_result = sqlc.sql("""
    SELECT COUNT(*) as count
    FROM tweets
    WHERE text LIKE '%EarnHistory%'
    AND lower(text) LIKE '%astros%' AND LOWER(text) NOT LIKE '%dodgers%'
""")

earnhistory_astros_result.show()

+-----+
|count|
+-----+
|19524|
+-----+



From the above two tweets, we can say that astros clearly are using the word earnhistory in their tweets more than the dodgers